# 14 - Knowledge Retrieval (RAG)
## LangChain 
- Let's now see an example end to end with LangChain
- This code demonstrates a RAG (Retrieval-Augmented Generation) pipeline using LangChain and LangGraph. 
- It starts by preparing a knowledge base from a text document, chunking it, and storing its embeddings in a Weaviate vector store for retrieval. The core of the RAG functionality is then defined using LangGraph's StateGraph, which orchestrates the flow between retrieve_documents_node and generate_response_node. The retrieve_documents_node fetches relevant information from the vector store based on the user's question. 
- Subsequently, the generate_response_node uses this retrieved context, along with a predefined prompt template, to generate a concise answer using an OpenAI LLM. Finally, the app.stream method allows for running queries through this compiled RAG pipeline, showcasing its ability to provide contextually relevant answers.

In [57]:
import os
import requests
from typing import List, Dict, Any, TypedDict

from langchain_core.documents import Document

from langchain.prompts import ChatPromptTemplate

from langchain_community.document_loaders import TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Weaviate
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langgraph.graph import StateGraph, END
import weaviate
from weaviate.embedded import EmbeddedOptions
import dotenv
from langchain_core.messages import ai

# Load environment variables (e.g., OPENAI_API_KEY)
dotenv.load_dotenv()

True

In [58]:
# --- 1. Data Preparation (Preprocessing) ---
# Load data
url = "https://github.com/langchain-ai/langchain/blob/master/docs/docs/how_to/state_of_the_union.txt"

res = requests.get(url)

with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()

In [59]:
# Chunk documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Embed and store chunks in Weaviate
client = weaviate.Client(embedded_options=EmbeddedOptions())

vectorstore = Weaviate.from_documents(
    client=client, documents=chunks, embedding=OpenAIEmbeddings(), by_text=False
)

# Define the retriever
retriever = vectorstore.as_retriever()

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

Created a chunk of size 3173, which is longer than the specified 1000
Created a chunk of size 13732, which is longer than the specified 1000
Created a chunk of size 1528, which is longer than the specified 1000
Created a chunk of size 1610, which is longer than the specified 1000
Created a chunk of size 2550, which is longer than the specified 1000
Created a chunk of size 2458, which is longer than the specified 1000
Created a chunk of size 1448, which is longer than the specified 1000
Created a chunk of size 1390, which is longer than the specified 1000
Created a chunk of size 1187, which is longer than the specified 1000
Created a chunk of size 1426, which is longer than the specified 1000
Created a chunk of size 1048, which is longer than the specified 1000
Created a chunk of size 1419, which is longer than the specified 1000
Created a chunk of size 1517, which is longer than the specified 1000
Created a chunk of size 1295, which is longer than the specified 1000
Created a chunk of 

Started /Users/mahtabsyed/.cache/weaviate-embedded: process ID 53943


{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2025-06-03T14:55:41+10:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2025-06-03T14:55:41+10:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2025-06-03T14:55:41+10:00"}
adding route GET /v1/schema/{className} "schema.objects.get"
operation: spec.Operation{VendorExtensible:spec.VendorExtensible{Extensions:spec.Extensions{"x-serviceIds":[]interface {}{"weaviate.local.get.meta"}}}, OperationProps:spec.OperationProps{Description:"", Consumes:[]string(nil), Produces:[]string(nil), Schemes:[]string(nil), Tags:[]string{"schema"}, Summary:"Get a single class from the schema", ExternalDocs:(*spec.ExternalDocumentation)(nil), ID:"schema.objects.get", Deprecated:false, Se

In [60]:
# --- 2. Define the State for LangGraph ---


class RAGGraphState(TypedDict):
    question: str
    documents: List[Document]
    generation: str


# --- 3. Define the Nodes (Functions) ---


def retrieve_documents_node(state: RAGGraphState) -> RAGGraphState:
    """Retrieves documents based on the user's question."""
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question, "generation": ""}


def generate_response_node(state: RAGGraphState) -> RAGGraphState:
    """Generates a response using the LLM based on retrieved documents."""
    question = state["question"]
    documents = state["documents"]

    # Prompt template from the PDF
    template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
    prompt = ChatPromptTemplate.from_template(template)

    # Format the context from the documents
    context = "\n\n".join([doc.page_content for doc in documents])

    # Create the RAG chain
    rag_chain = prompt | llm | StrOutputParser()

    # Invoke the chain
    generation = rag_chain.invoke({"context": context, "question": question})
    return {"question": question, "documents": documents, "generation": generation}

In [61]:
# --- 4. Build the LangGraph Graph ---

workflow = StateGraph(RAGGraphState)

# Add nodes
workflow.add_node("retrieve", retrieve_documents_node)
workflow.add_node("generate", generate_response_node)

# Set the entry point
workflow.set_entry_point("retrieve")

# Add edges (transitions)
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

# Compile the graph
app = workflow.compile()

In [62]:
# --- 5. Run the RAG Application ---

# if __name__ == "__main__":
# print("\n--- Running RAG Query ---")
# query = "What did the president say about Justice Breyer. Answer in less than 100 words."
# inputs = {"question": query}
# for s in app.stream(inputs):
#     print(s)


# print("\n--- Running another RAG Query ---")
# query_2 = "What did the president say about the economy?"
# inputs_2 = {"question": query_2}
# for s in app.stream(inputs_2):
#     print(s)

In [63]:
if __name__ == "__main__":
    print("\n--- Running RAG Query ---")
    query = "What did the president say about JJustice Breyer. Answer in less than 100 words."
    inputs = {"question": query}

    result = app.invoke(inputs)
    print("\nAnswer:\n", result["generation"])


--- Running RAG Query ---


looking up route for POST /v1/graphql
got a router for POST
got a router for HEAD
got a router for DELETE
got a router for PUT
got a router for PATCH
got a router for GET
found a route for POST /v1/graphql with 1 parameters
validating content type for "application/json" against [application/json, application/yaml]
responding to POST /v1/graphql with produces: [application/json]
offers: [application/json]
[POST /v1/graphql] set response format "application/json" in context
[POST /v1/graphql] negotiated response format "application/json"



Answer:
 The president honored Justice Stephen Breyer during his address, acknowledging him as a dedicated public servant, Army veteran, and constitutional scholar. He expressed gratitude for Breyer's service and highlighted the importance of nominating a successor, which he had done by nominating Judge Ketanji Brown Jackson. The president emphasized that Jackson would continue Breyer's legacy of excellence on the Supreme Court.
